## Titanic Kaggle Analysis

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
from sklearn.linear_model import LogisticRegression 
from sklearn.metrics import classification_report as report
from sklearn.model_selection import train_test_split, cross_val_score, ShuffleSplit
from statsmodels.multivariate.pca import PCA
#%matplotlib notebook

In [3]:
def massageData(df):
    df.Cabin=df.Cabin.fillna('N')
    df['CabinDeck']=df.Cabin.apply(lambda y: y[0])
    df['HasAge'] = df.Age.notnull().apply(lambda y: 1 if y else 0)
    df['HasCabin'] = df.Cabin.apply(lambda y: 0 if y=='N' else 1)
    df['parchMale0']=df.apply(lambda row: 1 if (row['Sex']=='male' and row['Parch']==0) else 0,axis=1)
    df['parchMale']=df.apply(lambda row: 1 if (row['Sex']=='male' and row['Parch']>0) else 0, axis=1)
    df['sibspMale0']=df.apply(lambda row: 1 if (row['Sex']=='male' and row['SibSp']==0) else 0, axis=1)
    df['sibspMale']=df.apply(lambda row: 1 if (row['Sex']=='male' and row['SibSp']>0) else 0, axis=1)
    df['parchFemale0']=df.apply(lambda row: 1 if (row['Sex']=='female' and row['Parch']==0) else 0,axis=1)
    df['parchFemale']=df.apply(lambda row: 1 if (row['Sex']=='female' and row['Parch']>0) else 0, axis=1)
    df['sibspFemale0']=df.apply(lambda row: 1 if (row['Sex']=='female' and row['SibSp']==0) else 0, axis=1)
    df['sibspFemale']=df.apply(lambda row: 1 if (row['Sex']=='female' and row['SibSp']>0) else 0, axis=1)
    df['pcssMale0']=df.apply(lambda row: 1 if (row['Sex']=='male' and (row['Parch']==0 and row['SibSp']==0)) else 0,axis=1)
    df['pcssMale']=df.apply(lambda row: 1 if (row['Sex']=='male' and (row['Parch']>0 or row['SibSp']>0)) else 0, axis=1)
    df['pcssFemale0']=df.apply(lambda row: 1 if (row['Sex']=='female' and (row['SibSp']==0 and row['Parch']==0)) else 0, axis=1)
    df['pcssFemale']=df.apply(lambda row: 1 if (row['Sex']=='female' and (row['SibSp']>0 or row['Parch']>0)) else 0, axis=1)
    df.Age = df.Age.fillna(0)
    return df
def getDeckDummies(df):
    #deckX = pd.get_dummies(df.CabinDeck,prefix='deck')
    #if ('deck_T' not in deckX.columns):
    #    deckX = deckX.join(pd.Series(np.zeros(deckX.shape[0]),index=deckX.index,name='deck_T'))
    deckX = df['HasCabin']
    return deckX
def getEmbarkDummies(df):
    embarkX = pd.get_dummies(df.Embarked,prefix='embark')
    return embarkX
def getPclassDummies(df):
    pclassX = pd.get_dummies(df.Pclass,prefix='class')
    return pclassX
def getSexX(df):
    return df[['pcssMale0','pcssMale','pcssFemale0','pcssFemale']]
def getAgeX(df):
    return df[['Age','HasAge']]
def getFullModelX(df):
    deckX = getDeckDummies(df)
    embarkX = getEmbarkDummies(df)
    pclassX = getPclassDummies(df)
    sexX = getSexX(df)
    ageX = getAgeX(df)
    fareX = df.Fare
    return pd.concat([deckX,embarkX,pclassX,sexX,ageX,fareX],axis=1)
#    return pd.concat([deckX,pclassX,sexX,ageX,fareX],axis=1)



#### Data key

In [4]:
#survival  Survival: 0 = No, 1 = Yes
#pclass    Ticket class: 1 = 1st, 2 = 2nd, 3 = 3rd
#sex       Sex
#Age       Age in years
#sibsp     # of siblings / spouses aboard the Titanic
#parch     # of parents / children aboard the Titanic
#ticket    Ticket number
#fare      Passenger fare
#cabin     Cabin number
#embarked  Port of Embarkation: C = Cherbourg, Q = Queenstown, S = Southampton

### Fullmodel training and predictions

In [5]:
trainDF=massageData(pd.read_csv('train.csv',index_col=0))
trainX=getFullModelX(trainDF)
trainY=trainDF.Survived

In [6]:
testDF=massageData(pd.read_csv('test.csv',index_col=0))
testX=getFullModelX(testDF)

In [33]:
#%%capture
# prevent output
X=sm.add_constant(trainX)
# Repeat until the Hessian is invertible (otherwise this breaks summary output)
#while True:
fullModel=sm.Logit(trainY,X).fit(method='bfgs',maxiter=1000)
#if (fullModel.normalized_cov_params.any().any()):
#    break

Optimization terminated successfully.
         Current function value: 0.439823
         Iterations: 94
         Function evaluations: 98
         Gradient evaluations: 98


/opt/conda/envs/pytorch-py35/lib/python3.5/site-packages/statsmodels/base/model.py:473: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  'available', HessianInversionWarning)


In [34]:
fullModel.bic

872.0654088508295

In [ ]:
print(report(trainY,fullModel.predict(X).apply(lambda y: 1 if y > 0.5 else 0)))

In [143]:
fullModel.bic

872.17439117742265

In [144]:
logreg = LogisticRegression(C=100,max_iter=100)
logreg.fit(X,trainY)
logreg.score(X,trainY)

0.8125701459034792

In [145]:
cv = ShuffleSplit(n_splits=3, test_size=0.3, random_state=590145)
cvs=cross_val_score(logreg,X,trainY,cv=cv)
print(cvs.mean(),"+/-",cvs.std())

0.832089552239 +/- 0.0219695543939


In [146]:
predictions=fullModel.predict(X).apply(lambda y: 1 if y>0.5 else 0)

In [147]:
f = open('titanic_predictions.csv','w')
f.write('PassengerId,Survived\n')
for i,p in zip(predictions.index,predictions):
    f.write(str(i)+','+str(p)+'\n')
f.close()

## Individual parameter significance tests

### Embarkment: Significant, but likely confounding variable with pClass, Fare, etc

In [56]:
embarkX = trainX[['embark_C','embark_Q','embark_S']]

In [57]:
model = sm.Logit(trainY,embarkX)

In [58]:
model=model.fit()
model.summary()

Optimization terminated successfully.
         Current function value: 0.650800
         Iterations 4


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:               Survived   No. Observations:                  891
Model:                          Logit   Df Residuals:                      888
Method:                           MLE   Df Model:                            2
Date:                Sat, 24 Mar 2018   Pseudo R-squ.:                 0.02269
Time:                        12:56:31   Log-Likelihood:                -579.86
converged:                       True   LL-Null:                       -593.33
                                        LLR p-value:                 1.420e-06
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
embark_C       0.2151      0.155      1.386      0.166      -0.089       0.519
embark_Q      -0.4490      0.234     -1.921      0.055      -0.907       0.009
embark_S      -0.6769      0.083     -8.119      0.000      -0.840      -0.513
==============================================================================
"""

In [59]:
data = trainX.join(trainY)
nC = sum(data.embark_C)
nQ = sum(data.embark_Q)
nS = sum(data.embark_S)
surviveCpercent = sum(data[data['embark_C']==1].Survived)/sum(data.embark_C)
surviveQpercent = sum(data[data['embark_Q']==1].Survived)/sum(data.embark_Q)
surviveSpercent = sum(data[data['embark_S']==1].Survived)/sum(data.embark_S)

In [60]:
#Survival percentages (by eye) with total 
#passengers number from that embarkment point
print("C: ",surviveCpercent," ",nC)
print("Q: ",surviveQpercent," ",nQ)
print("S: ",surviveSpercent," ",nS)

C:  0.553571428571   168
Q:  0.38961038961   77
S:  0.336956521739   644


In [166]:
trainX[['class_1','class_2','class_3','embark_S','embark_Q','embark_C','Fare']].corr()

,class_1,class_2,class_3,embark_S,embark_Q,embark_C,Fare
class_1,1.000000,-0.288585,-0.626738,-0.170379,-0.155342,0.296423,0.591711
class_2,-0.288585,1.000000,-0.565210,0.192061,-0.127301,-0.125416,-0.118557
class_3,-0.626738,-0.565210,1.000000,-0.009511,0.237449,-0.153329,-0.413333
embark_S,-0.170379,0.192061,-0.009511,1.000000,-0.496624,-0.778359,-0.166603
embark_Q,-0.155342,-0.127301,0.237449,-0.496624,1.000000,-0.148258,-0.117216
embark_C,0.296423,-0.125416,-0.153329,-0.778359,-0.148258,1.000000,0.269335
Fare,0.591711,-0.118557,-0.413333,-0.166603,-0.117216,0.269335,1.000000


### Sex: very significant

In [146]:
sexX = pd.get_dummies(trainDF.Sex)
model = sm.Logit(trainY,sexX).fit()
model.summary()

Optimization terminated successfully.
         Current function value: 0.515041
         Iterations 5


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:               Survived   No. Observations:                  891
Model:                          Logit   Df Residuals:                      889
Method:                           MLE   Df Model:                            1
Date:                Sat, 24 Mar 2018   Pseudo R-squ.:                  0.2266
Time:                        13:36:16   Log-Likelihood:                -458.90
converged:                       True   LL-Null:                       -593.33
                                        LLR p-value:                 2.020e-60
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
female         1.0566      0.129      8.191      0.000       0.804       1.309
male          -1.4571      0.106    -13.701      0.000      -1.666      -1.249
==============================================================================
"""

### Pclass: very significant for 1st and 3rd class

In [142]:
pclassX = trainX[['class_1','class_2','class_3']]

In [144]:
model = sm.Logit(trainY,pclassX)
model = model.fit(method='cg')
model.summary()

Optimization terminated successfully.
         Current function value: 0.607805
         Iterations: 5
         Function evaluations: 15
         Gradient evaluations: 15


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:               Survived   No. Observations:                  891
Model:                          Logit   Df Residuals:                      888
Method:                           MLE   Df Model:                            2
Date:                Sat, 24 Mar 2018   Pseudo R-squ.:                 0.08726
Time:                        13:35:52   Log-Likelihood:                -541.55
converged:                       True   LL-Null:                       -593.33
                                        LLR p-value:                 3.274e-23
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
class_1        0.5306      0.141      3.766      0.000       0.254       0.807
class_2       -0.1088      0.148     -0.737      0.461      -0.398       0.181
class_3       -1.1398      0.105    -10.822      0.000      -1.346      -0.933
==============================================================================
"""

### Fare: correlated with pclass, so also very significant

In [64]:
fareX = sm.add_constant(trainX.Fare)
model = sm.Logit(trainY,fareX)
model = model.fit()
model.summary()

Optimization terminated successfully.
         Current function value: 0.627143
         Iterations 6


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:               Survived   No. Observations:                  891
Model:                          Logit   Df Residuals:                      889
Method:                           MLE   Df Model:                            1
Date:                Sat, 24 Mar 2018   Pseudo R-squ.:                 0.05822
Time:                        12:56:32   Log-Likelihood:                -558.78
converged:                       True   LL-Null:                       -593.33
                                        LLR p-value:                 9.427e-17
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.9413      0.095     -9.895      0.000      -1.128      -0.755
Fare           0.0152      0.002      6.809      0.000       0.011       0.020
==============================================================================
"""

### Age & HasAge: Significant

In [65]:
ageX = trainX[['Age','HasAge']]
model = sm.Logit(trainY,sm.add_constant(ageX))
model = model.fit()
model.summary()

Optimization terminated successfully.
         Current function value: 0.659134
         Iterations 5


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:               Survived   No. Observations:                  891
Model:                          Logit   Df Residuals:                      888
Method:                           MLE   Df Model:                            2
Date:                Sat, 24 Mar 2018   Pseudo R-squ.:                 0.01018
Time:                        12:56:33   Log-Likelihood:                -587.29
converged:                       True   LL-Null:                       -593.33
                                        LLR p-value:                  0.002385
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.8771      0.165     -5.315      0.000      -1.200      -0.554
Age           -0.0110      0.005     -2.057      0.040      -0.021      -0.001
HasAge         0.8203      0.240      3.425      0.001       0.351       1.290
==============================================================================
"""

### Deck

In [170]:
deckX = pd.get_dummies(trainDF['CabinDeck'])
model = sm.Logit(trainY,deckX)
model = model.fit(method='ncg')
model.summary()

Optimization terminated successfully.
         Current function value: 0.610797
         Iterations: 7
         Function evaluations: 8
         Gradient evaluations: 14
         Hessian evaluations: 7


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:               Survived   No. Observations:                  891
Model:                          Logit   Df Residuals:                      882
Method:                           MLE   Df Model:                            8
Date:                Sat, 24 Mar 2018   Pseudo R-squ.:                 0.08277
Time:                        14:39:32   Log-Likelihood:                -544.22
converged:                       True   LL-Null:                       -593.33
                                        LLR p-value:                 9.882e-18
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
A             -0.1335      0.518     -0.258      0.796      -1.148       0.881
B              1.0704      0.335      3.200      0.001       0.415       1.726
C              0.3773      0.265      1.424      0.155      -0.142       0.897
D              1.1394      0.406      2.805      0.005       0.343       1.936
E              1.0986      0.408      2.691      0.007       0.298       1.899
F              0.4700      0.570      0.824      0.410      -0.647       1.587
G                   0      1.000          0      1.000      -1.960       1.960
N             -0.8480      0.083    -10.184      0.000      -1.011      -0.685
T             -5.1954     13.508     -0.385      0.701     -31.670      21.279
==============================================================================
"""

In [171]:
deckX.join(trainX.Fare).corr()

,A,B,C,D,E,F,G,N,T,Fare
A,1.000000,-0.030880,-0.034846,-0.025663,-0.025256,-0.015923,-0.008787,-0.240136,-0.004386,0.019549
B,-0.030880,1.000000,-0.062841,-0.046280,-0.045547,-0.028715,-0.015847,-0.433053,-0.007910,0.386297
C,-0.034846,-0.062841,1.000000,-0.052225,-0.051398,-0.032403,-0.017883,-0.488683,-0.008926,0.364318
D,-0.025663,-0.046280,-0.052225,1.000000,-0.037852,-0.023864,-0.013170,-0.359896,-0.006574,0.098878
E,-0.025256,-0.045547,-0.051398,-0.037852,1.000000,-0.023486,-0.012961,-0.354194,-0.006470,0.053717
F,-0.015923,-0.028715,-0.032403,-0.023864,-0.023486,1.000000,-0.008171,-0.223299,-0.004079,-0.033093
G,-0.008787,-0.015847,-0.017883,-0.013170,-0.012961,-0.008171,1.000000,-0.123234,-0.002251,-0.025180
N,-0.240136,-0.433053,-0.488683,-0.359896,-0.354194,-0.223299,-0.123234,1.000000,-0.061513,-0.482075
T,-0.004386,-0.007910,-0.008926,-0.006574,-0.006470,-0.004079,-0.002251,-0.061513,1.000000,0.002224
Fare,0.019549,0.386297,0.364318,0.098878,0.053717,-0.033093,-0.025180,-0.482075,0.002224,1.000000


### Alternate model: Distinguish between has a cabin or not

In [151]:
hasCabinX = trainX[['deck_N']]
model = sm.Logit(trainY,hasCabinX).fit()
model.summary()

Optimization terminated successfully.
         Current function value: 0.629608
         Iterations 5


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:               Survived   No. Observations:                  891
Model:                          Logit   Df Residuals:                      890
Method:                           MLE   Df Model:                            0
Date:                Sat, 24 Mar 2018   Pseudo R-squ.:                 0.05452
Time:                        13:38:06   Log-Likelihood:                -560.98
converged:                       True   LL-Null:                       -593.33
                                        LLR p-value:                       nan
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
deck_N        -0.8480      0.083    -10.184      0.000      -1.011      -0.685
==============================================================================
"""

### Nonzero siblings/spouses

In [153]:
sibsp0X = trainDF.SibSp.apply(lambda y: 0 if y>0 else 1)
sibsp0X.name = 'SibSp0'
sibsp0X = pd.concat([sibsp0X,trainDF.SibSp.apply(lambda y: 1 if y>0 else 0)],axis=1)
model = sm.Logit(trainY,sibsp0X)
model = model.fit(method='newton')
model.summary()

Optimization terminated successfully.
         Current function value: 0.659268
         Iterations 4


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:               Survived   No. Observations:                  891
Model:                          Logit   Df Residuals:                      889
Method:                           MLE   Df Model:                            1
Date:                Sat, 24 Mar 2018   Pseudo R-squ.:                0.009977
Time:                        13:41:49   Log-Likelihood:                -587.41
converged:                       True   LL-Null:                       -593.33
                                        LLR p-value:                 0.0005801
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
SibSp0        -0.6393      0.085     -7.496      0.000      -0.807      -0.472
SibSp         -0.1345      0.119     -1.129      0.259      -0.368       0.099
==============================================================================
"""

### Nonzero parents or children

In [152]:
parch0X = trainDF.Parch.apply(lambda y: 0 if y>0 else 1)
parch0X.name = 'Parch0'
parch0X = pd.concat([parch0X,trainDF.Parch.apply(lambda y: 1 if y>0 else 0)],axis=1)
#parch0X = PCA(parch0X,method='eig').factors
model = sm.Logit(trainY,parch0X)
model = model.fit()
model.summary()

Optimization terminated successfully.
         Current function value: 0.655251
         Iterations 4


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:               Survived   No. Observations:                  891
Model:                          Logit   Df Residuals:                      889
Method:                           MLE   Df Model:                            1
Date:                Sat, 24 Mar 2018   Pseudo R-squ.:                 0.01601
Time:                        13:41:39   Log-Likelihood:                -583.83
converged:                       True   LL-Null:                       -593.33
                                        LLR p-value:                 1.308e-05
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Parch0        -0.6470      0.081     -8.002      0.000      -0.806      -0.489
Parch          0.0470      0.137      0.343      0.732      -0.222       0.316
==============================================================================
"""

### Test alternate model: Distinguish effect of parents/children, spouses/siblings between male/female

In [69]:
spX = trainDF[['parchMale0','parchMale','sibspMale0','sibspMale','parchFemale0','parchFemale','sibspFemale0','sibspFemale']]
spX = PCA(spX).factors
model = sm.Logit(trainY,spX)
model = model.fit_regularized()
model.summary()

Optimization terminated successfully.    (Exit mode 0)
            Current function value: 0.505195405863583
            Iterations: 127
            Function evaluations: 127
            Gradient evaluations: 127


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:               Survived   No. Observations:                  891
Model:                          Logit   Df Residuals:                      883
Method:                           MLE   Df Model:                            7
Date:                Sat, 24 Mar 2018   Pseudo R-squ.:                  0.2413
Time:                        12:56:34   Log-Likelihood:                -450.13
converged:                       True   LL-Null:                       -593.33
                                        LLR p-value:                 4.848e-58
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
comp_0      1086.5823   1236.854      0.879      0.380   -1337.607    3510.771
comp_1      -607.4048    415.100     -1.463      0.143   -1420.985     206.175
comp_2       904.9806    927.377      0.976      0.329    -912.645    2722.606
comp_3       133.4959   1093.182      0.122      0.903   -2009.102    2276.094
comp_4       443.0046   1482.397      0.299      0.765   -2462.441    3348.450
comp_5      1418.4648   1806.378      0.785      0.432   -2121.972    4958.901
comp_6      -707.8940   1325.906     -0.534      0.593   -3306.622    1890.834
comp_7      -304.8062    442.970     -0.688      0.491   -1173.012     563.400
==============================================================================
"""

In [70]:
sexX=pd.get_dummies(trainDF.Sex)
X=spX.join(deckX).join(ageX).join(pclassX).join(fareX).join(embarkX)
#X=spX.join(trainDF.deckX).join(trainX.N).join(ageX).join(pclassX).join(fareX).join(embarkX)

In [71]:
cv = ShuffleSplit(n_splits=3, test_size=0.3, random_state=314159)
cvs=cross_val_score(logreg,X,trainY,cv=cv)
print(cvs.mean(),"+/-",cvs.std())

0.803482587065 +/- 0.00879486046252


In [72]:
logreg.fit(X,trainY)
logreg.score(X,trainY)

0.81818181818181823

In [73]:
df=massageData(pd.read_csv('train.csv',index_col=0))

In [74]:
print(sum(df[df.parchMale0==1].Survived)/sum(df.parchMale0)," ",sum(df.parchMale0))
print(sum(df[df.parchMale==1].Survived)/sum(df.parchMale), " " , sum(df.parchMale))
print(sum(df[df.sibspMale0==1].Survived)/sum(df.sibspMale0), " " , sum(df.sibspMale0))
print(sum(df[df.sibspMale==1].Survived)/sum(df.sibspMale), " " , sum(df.sibspMale))
print(sum(df[df.parchFemale0==1].Survived)/sum(df.parchFemale0), " " , sum(df.parchFemale0))
print(sum(df[df.parchFemale==1].Survived)/sum(df.parchFemale), " " , sum(df.parchFemale))
print(sum(df[df.sibspFemale0==1].Survived)/sum(df.sibspFemale0), " " , sum(df.sibspFemale0))
print(sum(df[df.sibspFemale==1].Survived)/sum(df.sibspFemale), " " , sum(df.sibspFemale))

0.165289256198   484
0.311827956989   93
0.168202764977   434
0.251748251748   143
0.788659793814   194
0.666666666667   120
0.787356321839   174
0.685714285714   140


In [75]:
print(sum(df[df.pcssMale0==1].Survived)/sum(df.pcssMale0), " " , sum(df.pcssMale0))
print(sum(df[df.pcssMale==1].Survived)/sum(df.pcssMale), " " , sum(df.pcssMale))
print(sum(df[df.pcssFemale0==1].Survived)/sum(df.pcssFemale0), " " , sum(df.pcssFemale0))
print(sum(df[df.pcssFemale==1].Survived)/sum(df.pcssFemale), " " , sum(df.pcssFemale))

0.155717761557   411
0.271084337349   166
0.785714285714   126
0.712765957447   188


In [76]:
pcssX = df[['pcssMale0','pcssMale','pcssFemale0','pcssFemale']]
model = sm.Logit(trainY,pcssX).fit()
model.summary()

Optimization terminated successfully.
         Current function value: 0.508372
         Iterations 6


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:               Survived   No. Observations:                  891
Model:                          Logit   Df Residuals:                      887
Method:                           MLE   Df Model:                            3
Date:                Sat, 24 Mar 2018   Pseudo R-squ.:                  0.2366
Time:                        12:56:35   Log-Likelihood:                -452.96
converged:                       True   LL-Null:                       -593.33
                                        LLR p-value:                 1.468e-60
===============================================================================
                  coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------
pcssMale0      -1.6904      0.136    -12.426      0.000      -1.957      -1.424
pcssMale       -0.9891      0.175     -5.665      0.000      -1.331      -0.647
pcssFemale0     1.2993      0.217      5.984      0.000       0.874       1.725
pcssFemale      0.9089      0.161      5.639      0.000       0.593       1.225
===============================================================================
"""

### KNN Classifier

In [163]:
from sklearn.neighbors import KNeighborsClassifier

In [164]:
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(trainX, trainY,) 
knn.score(trainX, trainY)

0.83277216610549942